In [3]:
from skimage.measure import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from calc import *
from collections import OrderedDict

from ipywidgets import FloatProgress
from IPython.display import display

# Process Data w/ GFP

In [4]:
filename = 'exp249_a.txt'

with open(filename, 'a') as data_file:
    data_file.truncate(0)
    data_file.write('pos\tlabel\ttime\tx\ty\tarea\tis_cropped\tsignal\ttotal\tmean\tmedian\tmax\n')

In [ ]:
f = FloatProgress(min=0, max=33, description= 'Pos 1-33:')
display(f)

for pos in range(1, 33):
       
    frame = '%03d' % pos

    data_bf = np.load('exp249_bf/%s.npy' % frame)
    data_bf[data_bf>=0.5] = 1
    data_bf[data_bf<0.5] = 0
    data_bf = data_bf[:, :, :, 1]

    data_gfp = np.load('exp249_gfp/%s.npy' % frame)

    # 1st time frame
    pos_data = {}
    labels = {}
    regions = {}

    if pos == 6:
        time_0 = 5
        
    else:
        time_0 = 1
    labels[time_0] = {}
    
    regions = {}
    regions[time_0] = regionprops(label(data_bf[time_0-1]), data_gfp[time_0-1])
    median_pixel = np.median(regionprops(label(data_bf[time_0-1], background=1), data_gfp[time_0-1])[0].intensity_image)
    
    temp_list = [len(np.unique(data_gfp[time_0-1][n]))==1 for n in range(0, 312)]
    if True in temp_list:
        y_limit = temp_list.index(True)
    else:
        y_limit = 312
    
    for props in regions[time_0]:
        pos_data[len(pos_data)+1] = {time_0:[props, region_intensity(props, 9, median_pixel, y_limit)]}
        labels[time_0][props.label] = props.label

    # rest of time frames

    timeSpan = 241
    overlapThreshold = 0.4

    for time in range(time_0+1, timeSpan+1):

        labels[time] = {}
        
        gfp_img = data_gfp[time-1]
        temp_list = [len(np.unique(gfp_img[n]))==1 for n in range(0, 312)]
        if True in temp_list:
            y_limit = temp_list.index(True)
        else:
            y_limit = 312
            
        regions[time] = regionprops(label(data_bf[time-1]), gfp_img)

        for props2 in regions[time]:
            
            median_pixel = np.median(regionprops(label(data_bf[time-1], background=1), gfp_img)[0].intensity_image)

            for props1 in regions[time-1]:

                is_same = getOverlap(props1, props2, overlapThreshold)

                if is_same:

                    newLabel = labels[time-1][props1.label]
                    pos_data[newLabel][time] = [props2, region_intensity(props2, 9, median_pixel, y_limit)]
                    labels[time][props2.label] = newLabel
                    break

            if is_same != True:
                pos_data[len(pos_data)+1] = {time:[props2, region_intensity(props2, 9, median_pixel, y_limit)]}
                labels[time][props2.label] = len(pos_data)          

    # remove cells with only one time frame
    to_remove = [k for k in pos_data if len(pos_data[k]) < 15]
    for k in to_remove: del pos_data[k]
        
    sorted_data = dict(sorted(pos_data.items(), key = lambda item : len(item[1]), reverse=True))
    pos_data = {}
    for n, (key, value) in enumerate(sorted_data.items()):
        pos_data[n+1] = value
        
    with open(filename, 'a') as data_file:
        
        for label_id, label_data in pos_data.items():

            for time, props_data in label_data.items():

                r_data = props_data[0]
                i_data = props_data[1]
                data_file.write('%d\t%d\t%d\t%d\t%d\t%d\t%s\t%03f\t%d\t%03f\t%d\t%d\n' % (pos, label_id, time, r_data.centroid[1], r_data.centroid[0], r_data.area, i_data.is_cropped, i_data.signal_intensity, i_data.total_intensity, i_data.mean_intensity, i_data.median_intensity, i_data.max_intensity))
    
    f.value += 1
    
f.close()

print('Done processing.')


FloatProgress(value=0.0, description='Pos 1-5:', max=33.0)

## SIngle Position

In [ ]:
pos = 1
    
frame = '%03d' % pos

data_bf = np.load('exp249_bf/%s.npy' % frame)
data_bf[data_bf>=0.5] = 1
data_bf[data_bf<0.5] = 0
data_bf = data_bf[:, :, :, 1]

data_gfp = np.load('exp249_gfp/%s.npy' % frame)

# 1st time frame
pos_data = {}

time_0 = 
    
regions = {}
regions[time_0] = regionprops(label(data_bf[time_0-1]), data_gfp[time_0-1])

for props in regions[time_0]:
    pos_data[len(pos_data)+1] = {time_0:[props, region_intensity(props, 9)]}
        labels[time_0][props.label] = props.label

    # rest of time frames
        
    with open('exp249_t.txt', 'a') as data_file:
        
        for label_id, label_data in pos_data.items():

            for time, props_data in label_data.items():

                r_data = props_data[0]
                i_data = props_data[1]
                data_file.write('%d\t%d\t%d\t%d\t%d\t%d\t%03f\t%d\t%03f\t%d\t%d\n' % (pos, label_id, time, r_data.centroid[0], r_data.centroid[1], r_data.area, i_data.signal_intensity, i_data.total_intensity, i_data.mean_intensity, i_data.median_intensity, i_data.max_intensity))


## Print data parameters

In [14]:
x = pos_data[2].items()

In [31]:
for time, props in pos_data[1].items():
    print('Time:%d|'%time + 'X:%d|Y:%d|Area:%d|Intensity:%.3f'%(props[0].centroid[1], props[0].centroid[0], props[0].area, props[1]))
    

Time:1|X:60|Y:119|Area:890|Intensity:70.571
Time:2|X:60|Y:120|Area:918|Intensity:70.571
Time:3|X:60|Y:120|Area:915|Intensity:72.102
Time:4|X:60|Y:120|Area:923|Intensity:72.673
Time:5|X:59|Y:119|Area:910|Intensity:70.347
Time:6|X:59|Y:118|Area:917|Intensity:73.061
Time:7|X:59|Y:118|Area:909|Intensity:69.592
Time:8|X:58|Y:118|Area:912|Intensity:76.633
Time:9|X:58|Y:118|Area:925|Intensity:102.102
Time:10|X:58|Y:118|Area:959|Intensity:65.980
Time:11|X:58|Y:119|Area:946|Intensity:70.918
Time:12|X:59|Y:120|Area:939|Intensity:70.224
Time:13|X:59|Y:119|Area:935|Intensity:74.429
Time:14|X:59|Y:118|Area:931|Intensity:68.286
Time:15|X:59|Y:117|Area:919|Intensity:69.776
Time:16|X:59|Y:116|Area:940|Intensity:64.061
Time:17|X:59|Y:116|Area:936|Intensity:95.592
Time:18|X:58|Y:115|Area:955|Intensity:48.306
Time:19|X:58|Y:116|Area:950|Intensity:71.939
Time:20|X:59|Y:115|Area:970|Intensity:70.939
Time:21|X:58|Y:116|Area:995|Intensity:70.286
Time:22|X:57|Y:115|Area:970|Intensity:69.490
Time:23|X:57|Y:115